## Case study  - Uber Data Analysis

You have the uber drive data for a driver which captures the differnet aspects of driving behavior. You are a part of the project where you need to grill this data and report the important findings from the grilling and drilling exercise. You also need to provide some useful insights about the trip behaviour of a Uber driver.
Let us try to dive into the data with pandas and numpy by our side!

#### Dataset - 
There are two datasets to be used here. The dataset contains-

**Uberdrive.csv**
- Trip_Id - Id for the trip
- Start Date - the date and time of the start of the trip
- End Date - the date and time of the end of the trip
- Start Location - staring location of the trip 
- End Location  - location where the trip ended
- Purpose of drive - Purpose of the trip (Business, Personal, Meals, Errands, Meetings, Customer Support etc.)


**Uberdrive_Miles.csv**
- Trip_Id - Id for the trip
- Miles Driven  - Total miles driven between the start and the end of the trip

#### Objective-

The aim is to create a driver profile based on the below aspects on driving behavior.
- Name and number of all the unique start and stop points
- Popular start and stop points
- Rides with same start and stop points
- Starting point from which most miles have been driven
- Start- stop pairs that are most travelled in terms of distance
- busiest month in terms of number of drives and miles driven
- busiest day of the week
- peak hours
- most frequent trip category
- most frequent trip purpose
- miles driven per category and purpose
- percent composition of business miles vs personal miles
- time spent per category and purpose

----------------------
 #### Concepts To cover 
----------------------
- 1. <a href = #link1>Overview of the data at hand</a>
- 2. <a href = #link3>Filtering Data</a> 
- 3. <a href = #link2>Data profiling and the functions offered by pandas for understanding the data</a>
- 4. <a href = #link4>DateTime operations</a> 




In [ ]:
# Import the libraries 
import numpy as np
import pandas as pd
from datetime import datetime

### <a id = "link1"></a>Overview of the data

In [ ]:
# Read the Data 
df = pd.read_csv('uberdrive.csv')

In [ ]:
# View first 3 rows of data 
df.head(3) 

,Trip_Id,START_DATE*,END_DATE*,CATEGORY*,START*,STOP*,PURPOSE*
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies


In [ ]:
df_miles = pd.read_csv('UberDrive_Miles.csv')

In [ ]:
df_miles.tail(3)

,Trip_Id,MILES*
1152,1153,16.2
1153,1154,6.4
1154,1155,48.2


In [ ]:
# understand shape and size of data from Uberdrive
print(df.shape)
print(df.size)

(1155, 7)
8085


In [ ]:
# check info about data (includes column names, the number of non-null values in it, and data-type for each column.)
df.info()
print()
df_miles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1155 entries, 0 to 1154
Data columns (total 7 columns):
Trip_Id        1155 non-null int64
START_DATE*    1155 non-null object
END_DATE*      1155 non-null object
CATEGORY*      1155 non-null object
START*         1155 non-null object
STOP*          1155 non-null object
PURPOSE*       653 non-null object
dtypes: int64(1), object(6)
memory usage: 63.2+ KB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1155 entries, 0 to 1154
Data columns (total 2 columns):
Trip_Id    1155 non-null int64
MILES*     1155 non-null float64
dtypes: float64(1), int64(1)
memory usage: 18.1 KB


1. PURPOSE column has lots of missing values  - **We will talk about handling missing values in the upcoming weeks**
2. Some of the columns have a 1155 records while there are others with 653, why is that? Lets explore
3. We see some updates can be made in the column names, lets rename the columns

### Renaming columns

In [ ]:
# Approach 1
# Replace the * character from all the  columns
df.columns = df.columns.str.replace("*", "")

# Approach 2
#
# You can also rename the specific column names 
df_miles.rename(columns = {'MILES*':'MILES'}, inplace=True)
print(df.columns,"\n", df_miles.columns)

Index(['Trip_Id', 'START_DATE', 'END_DATE', 'CATEGORY', 'START', 'STOP',
       'PURPOSE'],
      dtype='object') 
 Index(['Trip_Id', 'MILES'], dtype='object')


**The column names were updated.**

### <a id = "link3"></a>Filtering dataframes
#### Using null values

In [ ]:
# shows the top 5 entries where PURPOSE is null
df[df.PURPOSE.isnull()].head(5)

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN
32,33,1/19/2016 9:09,1/19/2016 9:23,Business,Whitebridge,Lake Wellingborough,NaN
85,86,02-09-2016 10:54,02-09-2016 11:07,Personal,Whitebridge,Northwoods,NaN
86,87,02-09-2016 11:43,02-09-2016 11:50,Personal,Northwoods,Tanglewood,NaN
87,88,02-09-2016 13:36,02-09-2016 13:52,Personal,Tanglewood,Preston,NaN



#### Filtering out records based on conditions

In [ ]:
# Conditions within dataframe 
df_miles[df_miles['MILES'] > 30].head()

,Trip_Id,MILES
4,5,63.7
36,37,40.2
108,109,43.7
232,233,136.0
233,234,30.2


## <a id = "link2"></a>In a bid to create the driver profile, lets explore the data parameter wise - 

- 1.Destination - (starting and stopping)

- 2.Time - (hour of the day, day of week, month of year)

- 3.Grouping two parameters to get more insights

- 4.Category and Purpose

## 1. Destination
### Understanding  the start and stop points 
###### Through the feature, we will try to understand the below points of the driver profile. 
- Name and number of all the unique start and stop points
- Popular start and stop points
- Rides with same start and stop points
- Starting point from which most miles have been driven
- Start- stop pairs that are most travelled in terms of distance

**Let us handle these one by one**

**1. Name and Number of all unique start and stop points**

In [ ]:
# Get the unique starting point, unique destination
# names of unique start points
print(df['START'].unique())

['Fort Pierce' 'West Palm Beach' 'Cary' 'Jamaica' 'New York' 'Elmhurst'
 'Midtown' 'East Harlem' 'Flatiron District' 'Midtown East'
 'Hudson Square' 'Lower Manhattan' "Hell's Kitchen" 'Downtown' 'Gulfton'
 'Houston' 'Eagan Park' 'Morrisville' 'Durham' 'Farmington Woods'
 'Whitebridge' 'Lake Wellingborough' 'Fayetteville Street' 'Raleigh'
 'Hazelwood' 'Fairmont' 'Meredith Townes' 'Apex' 'Chapel Hill'
 'Northwoods' 'Edgehill Farms' 'Tanglewood' 'Preston' 'Eastgate'
 'East Elmhurst' 'Jackson Heights' 'Long Island City' 'Katunayaka'
 'Unknown Location' 'Colombo' 'Nugegoda' 'Islamabad' 'R?walpindi'
 'Noorpur Shahan' 'Heritage Pines' 'Westpark Place' 'Waverly Place'
 'Wayne Ridge' 'Weston' 'East Austin' 'West University' 'South Congress'
 'The Drag' 'Congress Ave District' 'Red River District' 'Georgian Acres'
 'North Austin' 'Coxville' 'Convention Center District' 'Austin' 'Katy'
 'Sharpstown' 'Sugar Land' 'Galveston' 'Port Bolivar' 'Washington Avenue'
 'Briar Meadow' 'Latta' 'Jacksonville'

In [ ]:
print(df['START'].nunique())                           # or use can use the nunique function

177


In [ ]:
# Get the names of stopping destinations, unique destinations
# Names of unique stopping points
print(df['STOP'].unique())

['Fort Pierce' 'West Palm Beach' 'Palm Beach' 'Cary' 'Morrisville'
 'New York' 'Queens' 'East Harlem' 'NoMad' 'Midtown' 'Midtown East'
 'Hudson Square' 'Lower Manhattan' "Hell's Kitchen" 'Queens County'
 'Gulfton' 'Downtown' 'Houston' 'Jamestown Court' 'Durham' 'Whitebridge'
 'Lake Wellingborough' 'Raleigh' 'Umstead' 'Hazelwood' 'Westpark Place'
 'Meredith Townes' 'Leesville Hollow' 'Apex' 'Chapel Hill'
 'Williamsburg Manor' 'Macgregor Downs' 'Edgehill Farms' 'Northwoods'
 'Tanglewood' 'Preston' 'Walnut Terrace' 'Jackson Heights' 'East Elmhurst'
 'Midtown West' 'Long Island City' 'Jamaica' 'Unknown Location' 'Colombo'
 'Nugegoda' 'Katunayaka' 'Islamabad' 'R?walpindi' 'Noorpur Shahan'
 'Heritage Pines' 'Waverly Place' 'Wayne Ridge' 'Depot Historic District'
 'Weston' 'West University' 'South Congress' 'Arts District'
 'Congress Ave District' 'Red River District' 'The Drag'
 'Convention Center District' 'North Austin' 'Coxville' 'Katy' 'Alief'
 'Sharpstown' 'Sugar Land' 'Galveston' 'Port

In [ ]:
print(len(df['STOP'].unique()))            # count of unique stopping points

188


**2a. Identify popular start points - top 10**

In [ ]:
df['START'].value_counts().head(10)

Cary                201
Unknown Location    148
Morrisville          85
Whitebridge          68
Islamabad            57
Durham               37
Lahore               36
Raleigh              28
Kar?chi              27
Apex                 17
Name: START, dtype: int64

**2b. Identify popular stop destinations - top 10**

In [ ]:

df['STOP'].value_counts().head(10)

Cary                203
Unknown Location    149
Morrisville          84
Whitebridge          65
Islamabad            58
Durham               36
Lahore               36
Raleigh              29
Kar?chi              26
Apex                 17
Name: STOP, dtype: int64

**3.  Are there cases where the start and the stop location are the same  ?**

In [ ]:
df[df['START'] == df['STOP']].head(5)

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies
3,4,01-05-2016 17:31,01-05-2016 17:45,Business,Fort Pierce,Fort Pierce,Meeting
5,6,01-06-2016 17:15,01-06-2016 17:19,Business,West Palm Beach,West Palm Beach,Meal/Entertain


In [ ]:
df[df['START'] == df['STOP']].shape

(288, 7)

**288 trips have the same start and stop points**

**4.Starting point from which the most miles have been driven**

**In order to use the miles feature, let us now merge the two dataframes so that the all the information is in one dataframe.**
- using merge 


In [ ]:
df = pd.merge(df, df_miles, on = 'Trip_Id', how = 'left')
df.head(5)

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE,MILES
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain,5.1
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN,5.0
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies,4.8
3,4,01-05-2016 17:31,01-05-2016 17:45,Business,Fort Pierce,Fort Pierce,Meeting,4.7
4,5,01-06-2016 14:42,01-06-2016 15:49,Business,Fort Pierce,West Palm Beach,Customer Visit,63.7


**Let's now use groupby function to find the starting point from which the most miles have been driven**

In [ ]:
df.groupby('START')['MILES'].sum().sort_values(ascending = False ).head(10)

START
Unknown Location    1976.5
Cary                1791.3
Morrisville          671.7
Raleigh              433.0
Islamabad            401.2
Durham               384.4
Jacksonville         375.2
Latta                310.3
Asheville            287.7
Whitebridge          273.4
Name: MILES, dtype: float64

**5. Find the top10 start stop pair that have the most miles covered between them ever.**

# Let us drop the unknown locations
df2 = df[df['START'] != 'Unknown Location']             # Makes a new dataframe, which don't have "Unknown Location" as starting point
df2 = df2[df2['STOP'] != 'Unknown Location']            # Further updates the df2 dataframe, by removing "Unknown Location" as stopping point

In [ ]:
# Creating a dataframe with the top 10 most miles covered between a start stop pair

k3 = df2.groupby(['START','STOP'])['MILES'].sum().sort_values(ascending=False).head(10) 
k3= k3.reset_index() # flatten the dataframe 
k3['Start-Stop'] = k3['START'] + ' - ' + k3['STOP']
k3 = df2.groupby(['START','STOP'])['MILES'].sum().sort_values(ascending=False).head(10) 
k3= k3.reset_index() # flatten the dataframe 
k3['Start-Stop'] = k3['START'] + ' - ' + k3['STOP']
k3

,START,STOP,MILES,Start-Stop
0,Morrisville,Cary,395.7,Morrisville - Cary
1,Cary,Durham,390.0,Cary - Durham
2,Cary,Morrisville,380.0,Cary - Morrisville
3,Raleigh,Cary,365.7,Raleigh - Cary
4,Cary,Raleigh,336.5,Cary - Raleigh
5,Durham,Cary,334.4,Durham - Cary
6,Latta,Jacksonville,310.3,Latta - Jacksonville
7,Cary,Cary,255.9,Cary - Cary
8,Jacksonville,Kissimmee,201.0,Jacksonville - Kissimmee
9,Asheville,Mebane,195.9,Asheville - Mebane


**The most popular start to destination pair is Morrisville-Cary**

<a id = "link4"></a>
## 2. Start Date - End Date 
### Manipulating date & time objects
#### Lets explore the variables using the below points-
- busiest month in terms of number of drives and miles driven
- busiest day of the week and preferred start hour 
- peak hours

We will create more features for the trip data to be able to cater to above profile mappings

In [ ]:
df.head(3)

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE,MILES
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain,5.1
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN,5.0
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies,4.8


In [ ]:
df.dtypes

Trip_Id         int64
START_DATE     object
END_DATE       object
CATEGORY       object
START          object
STOP           object
PURPOSE        object
MILES         float64
dtype: object

In [ ]:
# Create columns by converting the start and end date into a datatime format
# You can also over write the same column - but for the sake of understanding the difference in formats, we create new columns

df['start_dt'] = pd.to_datetime(df['START_DATE'])
df['end_dt'] = pd.to_datetime(df['END_DATE'])

In [ ]:
df.head()             # Print first 5 rows of data.

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE,MILES,start_dt,end_dt
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain,5.1,2016-01-01 21:11:00,2016-01-01 21:17:00
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN,5.0,2016-01-02 01:25:00,2016-01-02 01:37:00
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies,4.8,2016-01-02 20:25:00,2016-01-02 20:38:00
3,4,01-05-2016 17:31,01-05-2016 17:45,Business,Fort Pierce,Fort Pierce,Meeting,4.7,2016-01-05 17:31:00,2016-01-05 17:45:00
4,5,01-06-2016 14:42,01-06-2016 15:49,Business,Fort Pierce,West Palm Beach,Customer Visit,63.7,2016-01-06 14:42:00,2016-01-06 15:49:00


In [ ]:
df.dtypes             # See how the dtype is different

Trip_Id                int64
START_DATE            object
END_DATE              object
CATEGORY              object
START                 object
STOP                  object
PURPOSE               object
MILES                float64
start_dt      datetime64[ns]
end_dt        datetime64[ns]
dtype: object

In [ ]:
# Create more columns by using the inbuilt functionalities of datatime module

df['start_day'] = df['start_dt'].dt.day
df['start_hour'] = df['start_dt'].dt.hour
df['start_month'] = df['start_dt'].dt.month
df['d_of_wk'] = df['start_dt'].dt.dayofweek   # Days encoded as 0-6  ( monday =0, Tue =1 .... )

In [ ]:
df.head().T

,0,1,2,3,4
Trip_Id,1,2,3,4,5
START_DATE,01-01-2016 21:11,01-02-2016 01:25,01-02-2016 20:25,01-05-2016 17:31,01-06-2016 14:42
END_DATE,01-01-2016 21:17,01-02-2016 01:37,01-02-2016 20:38,01-05-2016 17:45,01-06-2016 15:49
CATEGORY,Business,Business,Business,Business,Business
START,Fort Pierce,Fort Pierce,Fort Pierce,Fort Pierce,Fort Pierce
STOP,Fort Pierce,Fort Pierce,Fort Pierce,Fort Pierce,West Palm Beach
PURPOSE,Meal/Entertain,NaN,Errand/Supplies,Meeting,Customer Visit
MILES,5.1,5,4.8,4.7,63.7
start_dt,2016-01-01 21:11:00,2016-01-02 01:25:00,2016-01-02 20:25:00,2016-01-05 17:31:00,2016-01-06 14:42:00
end_dt,2016-01-01 21:17:00,2016-01-02 01:37:00,2016-01-02 20:38:00,2016-01-05 17:45:00,2016-01-06 15:49:00


In [ ]:
df['weekday'] = df['start_dt'].apply(lambda x : datetime.strftime(x,'%a'))  # ( or directly convert into the short form)

In [ ]:
df.head().T

,0,1,2,3,4
Trip_Id,1,2,3,4,5
START_DATE,01-01-2016 21:11,01-02-2016 01:25,01-02-2016 20:25,01-05-2016 17:31,01-06-2016 14:42
END_DATE,01-01-2016 21:17,01-02-2016 01:37,01-02-2016 20:38,01-05-2016 17:45,01-06-2016 15:49
CATEGORY,Business,Business,Business,Business,Business
START,Fort Pierce,Fort Pierce,Fort Pierce,Fort Pierce,Fort Pierce
STOP,Fort Pierce,Fort Pierce,Fort Pierce,Fort Pierce,West Palm Beach
PURPOSE,Meal/Entertain,NaN,Errand/Supplies,Meeting,Customer Visit
MILES,5.1,5,4.8,4.7,63.7
start_dt,2016-01-01 21:11:00,2016-01-02 01:25:00,2016-01-02 20:25:00,2016-01-05 17:31:00,2016-01-06 14:42:00
end_dt,2016-01-01 21:17:00,2016-01-02 01:37:00,2016-01-02 20:38:00,2016-01-05 17:45:00,2016-01-06 15:49:00


In [ ]:
df['cal_month'] =  df['start_dt'].apply(lambda x : datetime.strftime(x,'%b'))

In [ ]:
df.head().T

,0,1,2,3,4
Trip_Id,1,2,3,4,5
START_DATE,01-01-2016 21:11,01-02-2016 01:25,01-02-2016 20:25,01-05-2016 17:31,01-06-2016 14:42
END_DATE,01-01-2016 21:17,01-02-2016 01:37,01-02-2016 20:38,01-05-2016 17:45,01-06-2016 15:49
CATEGORY,Business,Business,Business,Business,Business
START,Fort Pierce,Fort Pierce,Fort Pierce,Fort Pierce,Fort Pierce
STOP,Fort Pierce,Fort Pierce,Fort Pierce,Fort Pierce,West Palm Beach
PURPOSE,Meal/Entertain,NaN,Errand/Supplies,Meeting,Customer Visit
MILES,5.1,5,4.8,4.7,63.7
start_dt,2016-01-01 21:11:00,2016-01-02 01:25:00,2016-01-02 20:25:00,2016-01-05 17:31:00,2016-01-06 14:42:00
end_dt,2016-01-01 21:17:00,2016-01-02 01:37:00,2016-01-02 20:38:00,2016-01-05 17:45:00,2016-01-06 15:49:00


**Now let us answer the questions above.**
<br>
<br>
**1. Busiest month in terms of number of drives and miles driven**

In [ ]:
#groupby calender months and count the number of drives
df.groupby('cal_month').count()['Trip_Id'].sort_values(ascending = False)           

cal_month
Dec    146
Aug    133
Nov    122
Feb    115
Mar    113
Jul    112
Jun    108
Oct    106
Jan     61
Apr     54
May     49
Sep     36
Name: Trip_Id, dtype: int64

**December appears to be the busiest month in terms of number of drives**

In [ ]:
#groupby calender months and count the number of drives
df.groupby('cal_month').sum()['MILES'].sort_values(ascending = False)           

cal_month
Oct    1810.0
Mar    1693.9
Aug    1335.5
Jul    1224.6
Apr    1113.0
Dec     981.3
Feb     908.2
Jun     842.8
Nov     816.9
Sep     601.8
Jan     512.9
May     363.8
Name: MILES, dtype: float64

**October appears to be the busiest month in terms of miles driven**

**2. Busiest day in terms of number of rides**

In [ ]:
# Which day did the driver get most drives? 

df.groupby(['weekday']).size()  
#note that .count() could also have been used. However, .size() makes it look more clean.

weekday
Fri    206
Mon    174
Sat    150
Sun    148
Thu    154
Tue    176
Wed    147
dtype: int64

**3. Peak hours ?**

In [ ]:
df.groupby('start_hour').size()             # The number of trips started for each hour.

start_hour
0     19
1      5
2      2
3      3
5      4
6      4
7     13
8     35
9     51
10    65
11    72
12    77
13    94
14    89
15    98
16    88
17    95
18    94
19    68
20    71
21    51
22    31
23    26
dtype: int64

**Looks like the peak hours seem to be between 13PM - 6PM**

#### For practice - figure out the trips that are starting and ending at the same time (0 minutes elapsed).<br>
The first step is in the cell below. Try to figure out the rest of the steps after this session. 
<br>

In [ ]:
df['diff'] = (df['end_dt'] - df['start_dt'])

This creates a timedelta datatype

Find the date time units in  https://docs.scipy.org/doc/numpy/reference/arrays.datetime.html

search for 'Datetime Units'

#### For practice- Exploring existing features to create new ones - Speed 
- Open for all of you to explore and figure out what all can be understood and derived from this feature

## 4. Category & Purpose
#### Explore the category and the purpose of the trips through
- Most frequent trip category
- Most frequent trip purpose
- Miles driven per category and purpose
- Percent composition of business miles vs personal miles
- time spent per category and purpose

**1. Most frequent trip category**

In [ ]:
df['CATEGORY'].value_counts()

Business    1078
Personal      77
Name: CATEGORY, dtype: int64

**Most trips are in the business category**

**2. Most frequent Purpose**

In [ ]:
#Purpose
df['PURPOSE'].value_counts()

Meeting            187
Meal/Entertain     160
Errand/Supplies    128
Customer Visit     101
Temporary Site      50
Between Offices     18
Moving               4
Airport/Travel       3
Charity ($)          1
Commute              1
Name: PURPOSE, dtype: int64

**Most trips are for meetings**

In [ ]:
#Average distance traveled for each activity
df.groupby('PURPOSE').mean()['MILES'].sort_values(ascending = False)

PURPOSE
Commute            180.200000
Customer Visit      20.688119
Meeting             15.247594
Charity ($)         15.100000
Between Offices     10.944444
Temporary Site      10.474000
Meal/Entertain       5.698125
Airport/Travel       5.500000
Moving               4.550000
Errand/Supplies      3.968750
Name: MILES, dtype: float64

**Now lets try to answer some questions from this data.**

**Question3**: How many miles were driven per category and purpose ?

**Question4**: What is percentage of business miles vs personal?

**Question5**: How much time was spend for drives per category and purpose?  - <i>for practice: you will need to create a time difference variable - answers will be shared through a notebook</i>


In [ ]:
#Question3: How many miles were driven per category and purpose ?
df.groupby('PURPOSE').sum()['MILES'].sort_values(ascending = False)

PURPOSE
Meeting            2851.3
Customer Visit     2089.5
Meal/Entertain      911.7
Temporary Site      523.7
Errand/Supplies     508.0
Between Offices     197.0
Commute             180.2
Moving               18.2
Airport/Travel       16.5
Charity ($)          15.1
Name: MILES, dtype: float64

In [ ]:
#Question3: How many miles were driven per category and purpose ?
df.groupby('CATEGORY').sum()['MILES'].sort_values(ascending = False)

CATEGORY
Business    11487.0
Personal      717.7
Name: MILES, dtype: float64

In [ ]:
#Question4: What is percentage of business miles vs personal?
df1 = df.groupby('CATEGORY').agg({'MILES':'sum'})
df1
df1.apply(lambda x: x/x.sum()*100).rename(columns = {'MILES':'% of Miles'})

,% of Miles
CATEGORY,
Business,94.119479
Personal,5.880521


## Profile Report - 
Through the exercise, we discussed the various aspects the driver profile for the uberdriver data given. The insights received were-

**Name and number of all the unique start and stop points**<br>
We found the unique start and stop points for the driver. We figured out the localities the driver is active in.

**Popular start and stop point**<br>
Cary has been the most popular start and stop point

**Rides with same start and stop points**<br>
288 such rides were found

**Starting point from which most miles have been driven**<br>
Unknown location followed by Cary

**Start- stop pairs that are most travelled in terms of distance**<br>
Morissville - Cary

**busiest month in terms of number of drives and miles driven**<br>
In terms of no of drives - December
In terms of miles driven - October

**busiest day of the week**<br>
Friday

**peak hours**<br>
1PM - 6PM

**most frequent trip category**<br>
Business

**most frequent trip purpose**<br>
Meeting

**miles driven per category and purpose**<br>
We figured these numbers out. Category wise Business and purpose wise meetings were leading in terms of miles driven

**percent composition of business miles vs personal miles**<br>
Business - 94%
Personal - 6%


## Summary  -
Through this exercise, we tried to check out the data analysis toolkit offered by pandas. We went to explore variables at hand, use groupby, implement datatime manipulation, explored possibility to create new features and various other operations on pandas dataframe.
We also had a sneak peek into the upcoming week's topic of visualization.

Learners are recommended to explore further on this building on the points discussed in the notebook.
Happy Learning!